In [1]:
#!pip install uszipcode
#pip install pyqt5==5.12.0

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes = True)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from uszipcode import SearchEngine

import time
import datetime
import dateutil.parser
#import dateutil
#from dateutil.parser import *


In [3]:
daily = pd.read_csv("beat19-collab-data\\versions\\2020-06-01\\daily-data.csv")
enrollment = pd.read_csv("beat19-collab-data\\versions\\2020-06-01\\enrollment-data.csv")

In [4]:
print(daily.info())
print("*********************************************")
print(enrollment.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39032 entries, 0 to 39031
Data columns (total 81 columns):
_id                                  39032 non-null object
rel_date                             39032 non-null int64
date                                 39032 non-null object
updated_at                           39032 non-null object
version                              39032 non-null int64
ble_prompt                           39032 non-null bool
ble_travel_prompt                    39032 non-null bool
ble_travel_outside                   14395 non-null float64
ble_travel_bike                      4649 non-null float64
ble_travel_bus                       4649 non-null float64
ble_travel_car                       4649 non-null float64
ble_travel_train                     4649 non-null float64
ble_travel_taxi                      4649 non-null float64
ble_travel_walk                      4649 non-null float64
ble_travel_time                      3209 non-null float64
ble_travel_

In [5]:
daily_ttl_records = len(daily)
daily_unique_ids = daily._id.nunique()

enrollment_ttl = len(enrollment)
unique_enrollment = enrollment._id.nunique()

print(f"number of records in daily df: {daily_ttl_records}" )
print(f"number of unique _ids in daily df': {daily_unique_ids}")
print()
print("*************************************************************")

print(f"number of records in enrollment df: {enrollment_ttl}" )
print(f"number of unique _ids in enrollment df': {unique_enrollment}")
print()

number of records in daily df: 39032
number of unique _ids in daily df': 2081

*************************************************************
number of records in enrollment df: 2905
number of unique _ids in enrollment df': 2905



**Average number of entries per participant:**

In [6]:
avg_time_steps = daily_ttl_records / daily_unique_ids
print(f"each unique id has an average of{avg_time_steps: .0f} records")

each unique id has an average of 19 records


**Columns with NaN values:**

In [7]:
cols = list(daily)
counter = 0
for i in cols:
    if daily[i].isna().any() ==True:  #daily[i] is a series
        counter += 1
        
print(f"there are {counter} out of 81 columns ({counter/len(daily.columns) * 100: .0f}%) in df daily with NaN values")
print(f"percent of column NaN values range from 60% to 100%")

there are 56 out of 81 columns ( 69%) in df daily with NaN values
percent of column NaN values range from 60% to 100%


**NaN percent per feature:**

In [8]:
percent_nan = daily.isna().sum() * 100 / len(daily)
missing_values = pd.DataFrame({'column_name' : daily.columns,
                              'percent_nan' : percent_nan})
missing_values.reset_index(drop=True, inplace = True)
missing_values.sort_values(by= "percent_nan")

,column_name,percent_nan
0,_id,0.000000
27,ble_care_covid_prompt,0.000000
32,med_prompt,0.000000
34,med_rx_prompt,0.000000
36,med_supp_prompt,0.000000
39,sym_prompt,0.000000
79,change_beta_blocker,0.000000
25,ble_care_flu_prompt,0.000000
51,sym_git_prompt,0.000000
66,sym_fever_prompt,0.000000


In [9]:
# remove NaN columns
missing_val_dict = dict(zip(missing_values["column_name"], missing_values["percent_nan"]))

to_drop = []
for k, v in missing_val_dict.items():
    if v >= 50:
        to_drop.append(k)
        
daily.drop(columns = to_drop, inplace = True)

In [10]:
# impute county based off zip

search = SearchEngine(simple_zipcode=False)

counties = []
for i in enrollment.zip_postcode:
    counties.append(search.by_zipcode(i).county)

enrollment.insert(10, "county", counties )

In [19]:
# view records missing county info; can we fix?
missing_county = enrollment[enrollment['county'].isnull()]
num_missing = len(missing_county)
print(f"Enrollment db cannot match counties to {num_missing} zip codes that were not entered accurately. This translates into 149 total time steps.")

Enrollment db cannot match counties to 14 zip codes that were not entered accurately. This translates into 149 total time steps.


In [12]:
# pull in county by id into daily df

data = daily.merge(enrollment[['_id','state', 'county']] , how = 'outer', on = '_id')
print(data.shape)

# drop records missing county
data = data[data['county'].notna()]
print(data.shape)

(39856, 27)
(39707, 27)


In [13]:
# change date and updated_at to type datetime
data["date"] = pd.to_datetime(data["date"])
data["updated_at"] = pd.to_datetime(data["updated_at"])

**Average number of entries per day per county:**

In [20]:
# calc average number of records per county per day
unique_counties = list(data.county.unique())
avg_visit_per_day_per_county = {}
for cnty in unique_counties:
    df = data[data["county"] == cnty ]
    ttl_dates = df["date"].count()
    unique_dates = len(list(df["date"].unique()))
    avg_daily_visits = ttl_dates / unique_dates
    avg_visit_per_day_per_county[cnty] = avg_daily_visits
    
sorted_avg_visits = sorted(avg_visit_per_day_per_county.items(), key = lambda x: x[1], reverse = True)
sorted_avg_visits

[('Santa Clara County', 50.04615384615385),
 ('San Mateo County', 26.5625),
 ('San Diego County', 16.266666666666666),
 ('Middlesex County', 14.875),
 ('San Francisco County', 14.661538461538461),
 ('Durham County', 14.476923076923077),
 ('Alameda County', 14.046153846153846),
 ('Los Angeles County', 13.921875),
 ('Jackson County', 12.904761904761905),
 ('Maricopa County', 12.375),
 ('Clark County', 11.571428571428571),
 ('Orange County', 10.938461538461539),
 ('Wake County', 9.738461538461538),
 ('Montgomery County', 9.396825396825397),
 ('King County', 9.107692307692307),
 ('Santa Cruz County', 8.553846153846154),
 ('Tulare County', 8.236363636363636),
 ('Contra Costa County', 6.923076923076923),
 ('Douglas County', 6.203125),
 ('Harris County', 5.87037037037037),
 ('Norfolk County', 5.73015873015873),
 ('Suffolk County', 5.5625),
 ('Cook County', 5.4375),
 ('Delaware County', 5.423728813559322),
 ('Fairfax County', 5.396825396825397),
 ('Pinellas County', 5.298245614035087),
 ('Mont